# 연구 및 기사 작성을 위한 에이전트 생성하기

멀티 에이전트 시스템(multi-agent systems)의 기본 개념을 소개하고, CrewAI 프레임워크에 대한 개요를 제공합니다.

In [1]:
# pip install crewai crewai-tools
# %pip install 'crewai[tools]'

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

In [3]:
import os
from crew_ai_utils import pretty_print_result, get_openai_api_key

import dotenv

dotenv.load_dotenv()
openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o'
os.environ["SERPER_API_KEY"] = os.getenv("SERPER_API_KEY")


In [4]:
from crewai_tools import SerperDevTool


# Loading Tools
search_tool = SerperDevTool()

/Users/frodo/Documents/project/frodo-generative-ai/frodo/crew-ai/deeplearning.ai/Multi_AI_Agent_Systems_with_crewAI/crewai-env/lib/python3.12/site-packages/pydantic/_internal/_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/Users/frodo/Documents/project/frodo-generative-ai/frodo/crew-ai/deeplearning.ai/Multi_AI_Agent_Systems_with_crewAI/crewai-env/lib/python3.12/site-packages/crewai_tools/tools/scrapegraph_scrape_tool/scrapegraph_scrape_tool.py:34: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://e

## 에이전트 생성
- 에이전트를 정의하고, `role` (역할), `goal` (목표) 및 `backstory` (배경 이야기) 정의
- LLM(Large Language Model)은 역할극을 할 때 성능이 더 좋은 것으로 알려져 있음

### Agent: Planner(콘텐츠 기획자)


In [5]:
planner = Agent(
    role="콘텐츠 기획자 (Content Planner)",
    goal="매력적이고 사실에 근거한 콘텐츠를 {topic}(주제)에 대해 기획합니다.",
    backstory="당신은 {topic}(주제)에 대한 블로그 기사 기획 작업을 하고 있습니다."
              "당신은 독자들에게 유용한 정보를 수집하여 독자들이 학습하고 "
              "정보에 입각한 결정을 내릴 수 있도록 돕습니다. "
              "당신의 작업은 콘텐츠 작가(Content Writer)가 이 주제에 대한 기사를 작성하는 데 기초가 됩니다.",
    allow_delegation=False,
    tools=[search_tool],
	verbose=True
)

### Agent: Writer(콘텐츠 작가)

In [6]:
writer = Agent(
    role="콘텐츠 작가 (Content Writer)",
    goal="주제({topic})에 대해 통찰력 있고 사실에 근거한 의견(opinion)을 작성합니다.",
    backstory="당신은 주제({topic})에 대한 새로운 의견(opinion)을 작성하고 있습니다. "
              "당신은 콘텐츠 기획자(Content Planner)의 작업, 즉 개요(outline)와 주제에 대한 관련 맥락(context)을 기반으로 글을 씁니다. "
              "당신은 콘텐츠 기획자(Content Planner)가 제공한 개요(outline)의 주요 목표와 방향을 따릅니다. "
              "또한 객관적이고 공정한 통찰력(insights)을 제공하고 콘텐츠 기획자(Content Planner)가 제공한 정보를 통해 이를 뒷받침합니다. "
              "당신은 당신의 진술이 객관적인 진술이 아닌 의견(opinions)일 때 의견(opinion)에서 인정합니다.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor(편집자)

In [7]:
editor = Agent(
    role="편집자 (Editor)",
    goal="제공된 블로그 게시물을 편집하여 조직의 글쓰기 스타일에 맞춥니다.",
    backstory="당신은 콘텐츠 작가(Content Writer)로부터 블로그 게시물을 받는 편집자입니다. "
              "당신의 목표는 블로그 게시물을 검토하여 저널리즘의 모범 사례를 따르고,"
              "의견이나 주장을 제시할 때 균형 잡힌 관점을 제공하고,"
              "가능한 경우 주요 논쟁 거리가 되는 주제나 의견을 피하는 것입니다.",
    allow_delegation=False,
    verbose=True
)

### Agent: Translator(번역자)

In [8]:
translator = Agent(
    role="번역자 (Translator)",
    goal="편집이 완료된 블로그 게시물을 요청한 언어로 정확하게 번역하여 사용자에게 제공합니다.",
    backstory="당신은 편집자(Editor)로부터 블로그 게시물을 받는 번역자입니다. "
              "당신은 다양한 언어에 능통한 숙련된 번역가이며, 특히 IT 도메인에 전문성이 있습니다. "
              "당신의 목표는 콘텐츠의 의미, 어조, 스타일을 유지하면서 텍스트를 정확하게 번역하는 것입니다. "
              "당신은 다양한 분야의 텍스트를 번역한 경험이 있으며, 각 언어의 문화적 뉘앙스를 이해하고 있습니다.",
    allow_delegation=False,
    verbose=True
)

## 작업 생성

- `task`, `description`, `expected_output`, `agent` 정의

### Task: Plan(콘텐츠 계획)

In [9]:
plan = Task(
    description=(
        "1. {topic}에 대한 최신 트렌드(latest trends), 주요 업체(key players) 및 주목할 만한 뉴스(noteworthy news)를 우선적으로 고려합니다.\n"
        "2. 대상 독자(target audience)의 관심사(interests)와 고충(pain points)을 고려하여 파악합니다.\n"
        "3. 서론(introduction), 핵심 내용(key points), 행동 촉구(call to action)를 포함한 상세한 콘텐츠 개요(content outline)를 작성합니다.\n"
        "4. SEO 키워드(SEO keywords)와 관련된 데이터(data) 또는 소스(sources)를 포함합니다.\n"
    ),
    expected_output="개요(outline), 대상 독자 분석(audience analysis), "
        "SEO 키워드(SEO keywords) 및 리소스(resources)를 포함하는 포괄적인 콘텐츠 계획 문서(comprehensive content plan document)입니다.",
    agent=planner,
)

### Task: Write(콘텐츠 작성)

In [10]:
write = Task(
    description=(
        "1. 콘텐츠 계획(content plan)을 활용하여 {topic}에 대한 설득력 있는 블로그 게시물(blog post)을 작성합니다.\n"
        "2. SEO 키워드(SEO keywords)를 자연스럽게 통합합니다.\n"
        "3. 섹션/소제목(Sections/Subtitles)이 매력적인 방식으로 적절하게 명명되었는지 확인합니다.\n"
        "4. 게시물이 매력적인 서론(introduction), 통찰력 있는 본문(insightful body) 및 요약 결론(summarizing conclusion)으로 구성되었는지 확인합니다.\n"
        "5. 문법적 오류(grammatical errors)가 없는지, 브랜드의 목소리(brand's voice)와 일치하는지 교정합니다(Proofread).\n"
    ),
    expected_output="잘 작성된 블로그 게시물(blog post)을 마크다운 형식(markdown format)으로 제공하며, 각 섹션은 2~3개의 단락(paragraphs)으로 구성되어 출판 준비가 완료되어야 합니다.",
    agent=writer,
)

### Task: Edit(콘텐츠 편집)

In [11]:
edit = Task(
    description=(
        "1. 주어진 블로그 게시물(blog post)에서 문법적 오류(grammatical errors)를 교정하고 브랜드의 목소리(brand's voice)와 일치하는지 확인합니다.\n"
    ),
    expected_output=(
        "잘 작성된 블로그 게시물(blog post)을 마크다운 형식(markdown format)으로 제공하며, "
        "각 섹션은 2~3개의 단락(paragraphs)으로 구성되어 출판 준비가 완료되어야 합니다."
    ),
    agent=editor,
)

### Task: Translate(번역)

In [13]:
translate = Task(
    description=(
        "문법적 오류(grammatical errors)를 교정하고 브랜드의 목소리(brand's voice)와 일치하는지 확인이 끝나면 일본어로 번역합니다.\n"
    ),
    expected_output=(
        "일본어로 번역된 블로그 게시물(blog post)을 마크다운 형식(markdown format)으로 제공합니다."
    ),
    agent=translator,
)

## Crew 구성

- 에이전트(Agents)로 Crew를 구성
- 해당 에이전트(agents)가 수행할 작업(tasks)을 전달
    - 이 간단한 예제에서는* 작업(tasks)이 순차적으로 수행 됨.(즉, 서로 종속적)  
    - 따라서 목록에서 작업(task)의 _순서(order)_ 가 중요함.
- `verbose=True`를 설정하면 실행에 대한 모든 로그를 볼 수 있음

In [14]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

crew = Crew(
    agents=[planner, writer, editor, translator],
    tasks=[plan, write, edit,translate],
    verbose=True
)

## TEST

In [15]:
crew.test(n_iterations=1, openai_model_name='gpt-4o')

2025-03-04 13:21:58,904 - 8665140224 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: 콘텐츠 기획자 (Content Planner)
## Task: 1. {topic}에 대한 최신 트렌드(latest trends), 주요 업체(key players) 및 주목할 만한 뉴스(noteworthy news)를 우선적으로 고려합니다.
2. 대상 독자(target audience)의 관심사(interests)와 고충(pain points)을 고려하여 파악합니다.
3. 서론(introduction), 핵심 내용(key points), 행동 촉구(call to action)를 포함한 상세한 콘텐츠 개요(content outline)를 작성합니다.
4. SEO 키워드(SEO keywords)와 관련된 데이터(data) 또는 소스(sources)를 포함합니다.



# Agent: 콘텐츠 기획자 (Content Planner)
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"{topic} latest trends 2023\"}"
## Tool Output: 
{'searchParameters': {'q': '{topic} latest trends 2023', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': 'Trending Topics: What Students Are Searching For In 2023 - Infobase', 'link': 'https://infobase.com/blog/trending-topics-what-students-are-searching-for-in-2023/', 'snippet': 'Trending Topics: What Students Are Searching for in 2023 · Climate Change · Study Skills & Critical Thinking · Social Media · Mental Health.'

                                    Tasks Scores                                     
                               (1-10 Higher is better)                               
┏━━━━━━━━━━━━━━━━━━━━┯━━━━━━━┯━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┯━━┯━━┓
┃ Tasks/Crew/Agents  │ Run 1 │ Avg. Total │ Agents                            │  │  ┃
┠────────────────────┼───────┼────────────┼───────────────────────────────────┼──┼──┨
┃ Task 1             │  8.0  │    8.0     │ - 콘텐츠 기획자 (Content Planner) │  │  ┃
┃                    │       │            │                                   │  │  ┃
┃ Task 2             │  9.0  │    9.0     │ - 콘텐츠 작가 (Content Writer)    │  │  ┃
┃                    │       │            │                                   │  │  ┃
┃ Task 3             │  9.0  │    9.0     │ - 편집자 (Editor)                 │  │  ┃
┃                    │       │            │                                   │  │  ┃
┃ Task 4             │  8.5  │    8.5     │ - 번역자 (Translator)             │  │  ┃
┃ Crew               │ 8.62  │    8.6     │                                   │  │  ┃
┃ Execution Time (s) │  42   │     42     │                                   │  │  ┃
┗━━━━━━━━━━━━━━━━━━━━┷━━━━━━━┷━━━━━━━━━━━━┷━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┷━━┷━━┛

## Training crew and agents

In [16]:
crew.train(n_iterations=1, filename='training.pkl')

2025-03-04 13:25:33,747 - 8665140224 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: 콘텐츠 기획자 (Content Planner)
## Task: 1. {topic}에 대한 최신 트렌드(latest trends), 주요 업체(key players) 및 주목할 만한 뉴스(noteworthy news)를 우선적으로 고려합니다.
2. 대상 독자(target audience)의 관심사(interests)와 고충(pain points)을 고려하여 파악합니다.
3. 서론(introduction), 핵심 내용(key points), 행동 촉구(call to action)를 포함한 상세한 콘텐츠 개요(content outline)를 작성합니다.
4. SEO 키워드(SEO keywords)와 관련된 데이터(data) 또는 소스(sources)를 포함합니다.



# Agent: 콘텐츠 기획자 (Content Planner)
## Thought: 생각: {topic}(주제)에 대한 최신 트렌드, 주요 업체, 뉴스 및 대상 독자의 관심사를 파악하기 위해 검색을 시작하겠습니다. 시스템에게 주제를 명시하지 않았으므로 이를 구체적으로 설정할 필요가 있습니다. 예를 들어, "인공지능"이라는 주제로 최신 정보와 데이터를 수집하도록 하겠습니다.
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"latest trends in artificial intelligence 2023\"}"
## Tool Output: 
{'searchParameters': {'q': 'latest trends in artificial intelligence 2023', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': "The state of AI in 2023: Generative AI's breakout year | McKinsey", 'link': 'https://www.mckinsey.com/ca

ValueError: Critical training data error: Missing fields (human_feedback) for agent dc916edc-9e34-4c32-9e69-8d32926faa14 in iteration 0.
This indicates a broken training process. Cannot proceed with evaluation.
Please check your training implementation.

## Running the Crew

In [19]:
result = crew.kickoff(inputs={"topic": "인공지능"})

 
[2025-03-04 13:03:46][INFO]: Planning the crew execution
# Agent: Crew Manager
## Task: 1. 인공지능에 대한 최신 트렌드(latest trends), 주요 업체(key players) 및 주목할 만한 뉴스(noteworthy news)를 우선적으로 고려합니다.
2. 대상 독자(target audience)의 관심사(interests)와 고충(pain points)을 고려하여 파악합니다.
3. 서론(introduction), 핵심 내용(key points), 행동 촉구(call to action)를 포함한 상세한 콘텐츠 개요(content outline)를 작성합니다.
4. SEO 키워드(SEO keywords)와 관련된 데이터(data) 또는 소스(sources)를 포함합니다.
1. **Research Phase**: Use the 'Search the internet with Serper' tool to compile data on the latest trends in AI, key players, and noteworthy news. Make sure to cover diverse sources to get a comprehensive view. Keep these insights organized and updated for integration into the content plan. 

2. **Audience Analysis**: Identify and analyze target audience details such as their professional backgrounds, locations, and main interests, as well as any pain points they may encounter in relation to AI. Use findings to tailor content topics and tones.

3. **Content Outline De

- Display the results of your execution as markdown in the notebook.

In [50]:
from IPython.display import Markdown
Markdown(result.raw)

```markdown
# The Pulse of AI: Trends and Transformations in 2023

## Introduction

Artificial Intelligence (AI) continues to reshape industries and pave the way for groundbreaking innovations. As we venture deeper into 2023, understanding the evolving landscape of AI is crucial for technology enthusiasts, business leaders, data scientists, and academics alike. This article delves into the prevailing AI trends, highlights the key players propelling this field forward, reviews noteworthy news, and addresses ongoing challenges and considerations that accompany AI integration.

## Latest AI Trends

In 2023, AI is characterized by significant advancements in machine learning techniques and a stronger focus on ethical AI development. The proliferation of generative AI models has led to a surge in creative applications, from art synthesis to automated content creation. Additionally, AI's role in developing personalized experiences continues to grow, redefining user interactions in various sectors.

Moreover, AI's integration into the Internet of Things (IoT) is creating smarter ecosystems, where interconnected devices offer seamless automation while enhancing efficiencies in sectors like healthcare and manufacturing. This trend is indicative of a broader shift toward more autonomous, intelligent systems that require minimal human intervention, paving the way for wider adoption across industries.

## Key Players in AI

Several organizations stand at the forefront of AI innovation. Companies like OpenAI, Google, Microsoft, and IBM are leading in AI research and development, consistently pushing the boundaries of what AI technologies can achieve. These key players are not only investing in technological advancements but also in developing frameworks for ethical AI use, which is a growing concern for many stakeholders.

In the startup ecosystem, emerging players are making significant strides by focusing on niche applications of AI. Startups are increasingly targeting specific industry problems, offering solutions that are both innovative and efficient. This diversification is essential in driving competition and fostering an environment where AI technologies can thrive.

## Noteworthy AI News

One of the most significant news stories this year has been the surge in AI-powered healthcare solutions that are revolutionizing patient diagnostics and treatment plans. Companies are developing AI tools that can predict disease outbreaks, customize treatment plans, and even assist in surgical procedures. These advancements hold the promise of enhancing patient outcomes significantly.

Another major development is the international focus on establishing AI regulatory frameworks. Governments worldwide are working to create standards that ensure AI technologies are safe, secure, and employed ethically. This ongoing dialogue is vital for addressing AI ethical implications and integration challenges that many organizations face today.

## Addressing Pain Points

Despite the rapid advancements, the integration of AI into existing systems poses several challenges. One primary concern is the cost considerations associated with implementing AI solutions, which can be prohibitively high for small and medium-sized enterprises. Additionally, AI security risks remain a pressing issue, with data privacy and protection requiring stringent measures to safeguard sensitive information.

Organizations are also grappling with the ethical implications of AI, such as bias in algorithmic decision-making and the potential displacement of jobs. Addressing these issues is essential for gaining public trust and ensuring the sustainable growth of AI technologies.

## Conclusion and Call to Action

The landscape of AI in 2023 is marked by innovation, challenge, and tremendous potential. For stakeholders across the spectrum, staying informed about AI trends, key players, and technological developments is crucial. We invite you to engage with these ongoing conversations, evaluate the opportunities AI presents for your industry, and collaborate in creating solutions that uphold ethical standards while driving progress.

Stay tuned for more insights and updates on emerging AI technologies by subscribing to our newsletter, and let's shape the future of AI together!
```

## load crew and agents

In [ ]:
TRAINING_DATA_FILE = “training_data.pkl”
TRAINED_AGENTS_DATA_FILE = “trained_agents_data.pkl”

In [ ]:
def crew(self) -> Crew:
    """Creates the Write Book Chapter Crew"""
    crew = Crew(
        agents=self.agents,
        tasks=self.tasks,
        process=Process.sequential,
        verbose=True,
        memory=True
    )
    output = self.agents[0]._use_trained_data(".")
    print(output)
    return crew

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [15]:
topic = "2025년 3월 2일 서울 날씨"
result = crew.kickoff(inputs={"topic": topic})

# Agent: 콘텐츠 작가 (Content Writer)
## Task: 1. 콘텐츠 계획(content plan)을 활용하여 2025년 3월 2일 서울 날씨에 대한 설득력 있는 블로그 게시물(blog post)을 작성합니다.
2. SEO 키워드(SEO keywords)를 자연스럽게 통합합니다.
3. 섹션/소제목(Sections/Subtitles)이 매력적인 방식으로 적절하게 명명되었는지 확인합니다.
4. 게시물이 매력적인 서론(introduction), 통찰력 있는 본문(insightful body) 및 요약 결론(summarizing conclusion)으로 구성되었는지 확인합니다.
5. 문법적 오류(grammatical errors)가 없는지, 브랜드의 목소리(brand's voice)와 일치하는지 교정합니다(Proofread).

# Agent: 콘텐츠 기획자 (Content Planner)
## Task: What are the SEO keywords we need to integrate naturally into the blog post about 2025년 3월 2일 서울 날씨, and can you provide more details on the brand's voice for consistency?


# Agent: 콘텐츠 기획자 (Content Planner)
## Final Answer: 
To effectively plan for the blog post about the weather in Seoul on March 2, 2025, incorporating relevant SEO keywords is crucial for maximizing search engine visibility. Here are some suggested keywords to integrate naturally into the content:

1. **2025 Seoul Weather**
2. **March 2 Weather Forecast Seoul**


In [17]:
from IPython.display import Markdown
Markdown(result.raw)

The comprehensive content plan for the 2025년 3월 2일 서울 날씨 blog post includes a detailed outline (introduction, key points, conclusion, call to action), an audience analysis outlining interests and pain points, a list of SEO keywords, and suggested resources for information and inspiration.